# Run FunSearch on Bin Packing
Five steps:
1. Implement 'LLM' interface.
2. Implement a 'SandBox' interface.
3. Prepare a 'specification'.
4. Prepare a dataset.
5. Start FunSearch.

## Preparation: download the project file from github. And update system path.

In [1]:
import shutil

folder_path = "/content/TSP_funsearch"
shutil.rmtree(folder_path)  # delete existing tsp if needed

In [1]:
!git clone https://github.com/Perkzi/TSP_funsearch.git

import sys

sys.path.append('/content/TSP_funsearch/')
print(sys.path)

fatal: destination path 'TSP_funsearch' already exists and is not an empty directory.
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/root/.ipython', '/content/TSP_funsearch/']


In [22]:
%cd /content/TSP_funsearch/
!git pull

/content/TSP_funsearch
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 1.05 KiB | 537.00 KiB/s, done.
From https://github.com/Perkzi/TSP_funsearch
   972efa5..4527588  master     -> origin/master
Updating 972efa5..4527588
Fast-forward
 TSP_algorithms/TSP_algorithms_evolution01.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


## 1. Implement LLM interface
Set the API's IP address according to your API provider (See line 65 in the following code).
```python
conn = http.client.HTTPSConnection("api.chatanywhere.com.cn")
```
You should prepare a 'key' for the LLM API. And fill them in the header (See line 76-80 in the following code).
```python
headers = {
    'Authorization': 'Bearer [put your key here, the key may start with "sk-..."]',
    'User-Agent': 'Apifox/1.0.0 (https://apifox.com)',
    'Content-Type': 'application/json'
}
```

In [2]:
import time
import json
import multiprocessing
from typing import Collection, Any
import http.client
import traceback
from implementation import sampler



def _trim_preface_of_body(sample: str) -> str:
    """Trim the redundant descriptions/symbols/'def' declaration before the function body.
    Please see my comments in sampler.LLM (in sampler.py).
    Since the LLM used in this file is not a pure code completion LLM, this trim function is required.

    -Example sample (function & description generated by LLM):
    -------------------------------------
    This is the optimized function ...
    def priority_v2(...) -> ...:
        return ...
    This function aims to ...
    -------------------------------------
    -This function removes the description above the function's signature, and the function's signature.
    -The indent of the code is preserved.
    -Return of this function:
    -------------------------------------
        return ...
    This function aims to ...
    -------------------------------------
    去掉def之前和自己的行
    """
    lines = sample.splitlines()
    func_body_lineno = 0
    find_def_declaration = False
    for lineno, line in enumerate(lines):
        # find the first 'def' statement in the given code
        if line[:3] == 'def':
            func_body_lineno = lineno
            find_def_declaration = True
            break
    if find_def_declaration:
        code = ''
        for line in lines[func_body_lineno + 1:]:
            code += line + '\n'
        return code
    return sample

from openai import OpenAI
import time
from typing import Collection
#from TSP_algorithms.additional_prompts import base_prompt as ap

# 1.choose a different additional_prompts
# 'base_prompt' 'step_by_step_prompts'  'multi_path_analysis_prompts'
# 'heuristic_meta_prompts' 'multi_phase_prompts' 'self_analysis_prompts'
from TSP_algorithms.additional_prompts3 import tsp_specific_prompt as ap

class LLMAPI:
    def __init__(self, samples_per_prompt: int, trim=True):
        API_KEY = "sk-dTP04yUylSiLIo7uF93084C07cA54c3eB64dD134Aa6fD274" # 设置 OpenAI API 密钥
        BASE_URL = 'https://api.bltcy.ai/v1'
        self.client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
        self._samples_per_prompt = samples_per_prompt
        self._trim = trim
        if ap:
            self._additional_prompt = ap
        else:
            self._additional_prompt = (
                "Complete a different and more complex Python function. "
                "Be creative and you can insert multiple if-else and for-loop in the code logic. "
                "Only output the Python code, no descriptions."
            )
        if type(self._additional_prompt) is not list:
          self._additional_prompt = [self._additional_prompt]

    def draw_samples(self, prompt: str) -> Collection[str]:
        """Returns multiple predicted continuations of `prompt`."""
        #print("draw-samples")
        return [self._draw_sample(prompt) for _ in range(self._samples_per_prompt)]

    def _draw_sample(self, content: str) -> str:
        max_retries = 3

        # whether have the pre prompt before main prompt
        if len(self._additional_prompt)<=1:
          prompt = '\n'.join([content, self._additional_prompt[0]])
        elif len(self._additional_prompt)==2:
          pre_prompt = '\n'.join([content, self._additional_prompt[0]])
          retries = 0
          pre_output = ''
          prompt = ''
          while retries < max_retries:
            try:
                #print("prompt")
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": pre_prompt}],
                    max_tokens=512,
                    stream=False,
                )
                print("pre_response",response)
                pre_output = response.choices[0].message.content
                # 将 第二个prompt中的 <insert optimization analysis here> 替换为分析结果。
                prompt = '\n'.join([content, self._additional_prompt[1].replace("<insert optimization analysis here>", pre_output)])
                break
                #return output
            except Exception as e:
                traceback.print_exc()
                print(f"Error occurred: {e}")
                retries += 1
                time.sleep(2)
        else:
          raise RuntimeError("list of additional_prompt should have length <= 2")

        # the main prompt
        retries = 0
        output = ''
        while retries < max_retries:
            try:
                #print("prompt")
                response = self.client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": prompt}],
                    max_tokens=512,
                    stream=False,
                )
                print("response",response)
                output = response.choices[0].message.content

                if self._trim:
                    output = _trim_preface_of_body(output)
                break
                #return output
            except Exception as e:
                traceback.print_exc()
                print(f"Error occurred: {e}")
                retries += 1
                time.sleep(2)
        if output == '':
            print("Failed")
            raise RuntimeError("Failed after multiple retries")
        return output

# test LLMAPI
'''
llm = LLMAPI(samples_per_prompt=4)
response = llm._draw_sample('hello')
print(response)
'''

"\nllm = LLMAPI(samples_per_prompt=4)\nresponse = llm._draw_sample('hello')\nprint(response)\n"

## 2. Implement a 'SandBox' interface

In [3]:
from implementation import evaluator
from implementation import evaluator_accelerate


class Sandbox(evaluator.Sandbox):
    """Sandbox for executing generated code. Implemented by RZ.

    RZ: Sandbox returns the 'score' of the program and:
    1) avoids the generated code to be harmful (accessing the internet, take up too much RAM).
    2) stops the execution of the code in time (avoid endless loop).
    """

    def __init__(self, verbose=False, numba_accelerate=True):
        """
        Args:
            verbose         : Print evaluate information.
            numba_accelerate: Use numba to accelerate the evaluation. It should be noted that not all numpy functions
                              support numba acceleration, such as np.piecewise().
        """
        self._verbose = verbose
        self._numba_accelerate = numba_accelerate

    def run(
            self,
            program: str,
            function_to_run: str,  # RZ: refers to the name of the function to run (e.g., 'evaluate')
            function_to_evolve: str,  # RZ: accelerate the code by decorating @numba.jit() on function_to_evolve.
            inputs: Any,  # refers to the dataset # self._inputs是数据集的dict，current_input是当前使用的数据集的key (名字str)
            test_input: str,  # refers to the current instance
            timeout_seconds: int,
            **kwargs  # RZ: add this
    ) -> tuple[Any, bool]:
        """Returns `function_to_run(test_input)` and whether execution succeeded.

        RZ: If the generated code (generated by LLM) is executed successfully,
        the output of this function is the score of a given program.
        RZ: PLEASE NOTE THAT this SandBox is only designed for bin-packing problem.
        返回 (function_to_run(test_input)运行结果，True)
        """
        dataset = inputs[test_input]
        try:
            # 当你有多个进程时，每个进程都会将它们的结果放入各自的 result_queue 中？
            result_queue = multiprocessing.Queue()
            # 创建一个新的进程 process，目标函数为 self._compile_and_run_function，并传递相应的参数。只并行了一个进程
            process = multiprocessing.Process(
                target=self._compile_and_run_function,
                args=(program, function_to_run, function_to_evolve, dataset, self._numba_accelerate, result_queue)
            )
            process.start()
            # 等待进程完成。如果进程在 timeout_seconds 时间内未完成，继续执行后续代码
            process.join(timeout=timeout_seconds)
            if process.is_alive():
                # 如果进程仍然在运行（超时未完成）
                # if the process is not finished in time, we consider the program illegal
                process.terminate()
                process.join()
                results = None, False
            else:
                # 如果进程已完成，从队列中获取结果。
                if not result_queue.empty():
                    results = result_queue.get_nowait()
                else:
                    results = None, False

            return results
        except:
            return None, False

    def _compile_and_run_function(self, program, function_to_run, function_to_evolve, dataset, numba_accelerate,
                                  result_queue):
        try:
            # optimize the code (decorate function_to_run with @numba.jit())
            # 加上 @numba.jit() 装饰器
            if numba_accelerate:
                program = evaluator_accelerate.add_numba_decorator(
                    program=program,
                    function_to_evolve=function_to_evolve
                )
            # compile the program, and maps the global func/var/class name to its address
            all_globals_namespace = {}
            # execute the program, map func/var/class to global namespace
            # exec 是一个内置的Python函数，用于动态执行Python代码。
            # program 是一个包含要执行代码的字符串。all_globals_namespace 是一个字典，用于保存执行代码期间创建的全局变量、函数和类
            exec(program, all_globals_namespace)
            # get the pointer of 'function_to_run'
            # 从 all_globals_namespace 中获取要运行的函数
            function_to_run = all_globals_namespace[function_to_run]
            # return the execution results
            results = function_to_run(dataset)
            # the results must be int or float
            if not isinstance(results, (int, float)):
                result_queue.put((None, False))
                return
            result_queue.put((results, True))
        except Exception:
            # if raise any exception, we assume the execution failed
            result_queue.put((None, False))

## 3. Prepare a 'specification'

In [5]:
# 2.choose a different initial algorithm
# 'TSP_algorithm_cheapest' 'TSP_algorithm_farthest' 'TSP_algorithm_nearest'‘TSP_algorithms_Initial''TSP_simple'
from TSP_algorithms import TSP_algorithm_cheapest3 as algo
specification = algo.specification

print(specification)



import numpy as np
import time
from pathlib import Path
from typing import Dict, List, Tuple

def tsp_evaluate(route: list[int], distances: np.ndarray) -> float:
    return (
        sum(distances[route[i], route[i + 1]] for i in range(len(route) - 1))
        + distances[route[-1], route[0]]
    )
    
# ---- small helpers ----------------------------------------------------- #

def _get_unvisited(n: int, visited: set[int]) -> np.ndarray:
    mask = np.ones(n, dtype=bool)
    mask[list(visited)] = False
    return np.nonzero(mask)[0]

def _arg_best(prio: np.ndarray) -> int:
    return int(np.argmax(prio))
    
@funsearch.evolve
def tsp_priority(
    start_idx: int,
    current_idx: int,
    candidate_idx: int,
    unvisited: np.ndarray,
    distances_row: np.ndarray
) -> float:
    """
    Calculate priority scores for all candidate cities at once.

    Args:
        start_idx: int: Index of the starting city (In the Traveling Salesman Problem (TSP),
            the solution requires

## 4. Prepare a dataset

In [6]:
!pip install tsplib95

In [7]:

from TSP_data import TSP_DataLoading
TSP_datasets = {}

instances_AlgorithmDevelop = TSP_DataLoading.build_funsearch_dataset(TSP_DataLoading.tsplib_data1)
instances_PerformanceTesting = TSP_DataLoading.build_funsearch_dataset(TSP_DataLoading.tsplib_data2)
instances_GeneralizationTesting = TSP_DataLoading.build_funsearch_dataset(TSP_DataLoading.tsplib_data3)

for name, instance in instances_AlgorithmDevelop.items():
  TSP_datasets[name] = {'section1':instance}

for name, instance in instances_PerformanceTesting.items():
  TSP_datasets[name] = {'section1':instance}

for name, instance in instances_GeneralizationTesting.items():
  TSP_datasets[name] = {'section1':instance}

# 3.选一个数据集
# 'berlin52', 'eil76' 'kroA100' 'pr124' 'd198' 'lin318' 'pcb442'
# name = 'berlin52'
# dataset_to_eval = {name: TSP_datasets[name]}
# print(dataset_to_eval)

# 选定你要的名字列表
names = ['berlin52']

# 批量构造 dataset_to_eval
dataset_to_eval = {name: TSP_datasets[name] for name in names}

# 打印确认
print(dataset_to_eval)


{'berlin52': {'section1': {'distances': array([[   0.        ,  666.10809934,  281.11385594, ...,  217.08293346,
         789.38267019, 1220.46097848],
       [ 666.10809934,    0.        ,  649.32657423, ...,  596.25917184,
        1421.55724471, 1716.04924172],
       [ 281.11385594,  649.32657423,    0.        , ...,  463.24939288,
         995.3140208 , 1483.59361012],
       ...,
       [ 217.08293346,  596.25917184,  463.24939288, ...,    0.        ,
         829.60834133, 1150.76061803],
       [ 789.38267019, 1421.55724471,  995.3140208 , ...,  829.60834133,
           0.        ,  624.81997407],
       [1220.46097848, 1716.04924172, 1483.59361012, ..., 1150.76061803,
         624.81997407,    0.        ]]), 'optimal_tour': None}}}


In [8]:
#specification = r'''
import numpy as np
import time
from pathlib import Path
from typing import Dict, List, Tuple

def tsp_evaluate(route: list[int], distances: np.ndarray) -> float:
    return (
        sum(distances[route[i], route[i + 1]] for i in range(len(route) - 1))
        + distances[route[-1], route[0]]
    )

# ---- small helpers ----------------------------------------------------- #

def _get_unvisited(n: int, visited: set[int]) -> np.ndarray:
    mask = np.ones(n, dtype=bool)
    mask[list(visited)] = False
    return np.nonzero(mask)[0]

def _arg_best(prio: np.ndarray) -> int:
    return int(np.argmax(prio))

# @funsearch.evolve
def tsp_priority(
    start_idx: int,
    current_idx: int,
    candidate_idx: int,
    unvisited: List[int],
    distances_row: np.ndarray
) -> float:
    """
    Calculate the priority score for choosing candidate_idx as the next city.
    Higher score means higher priority.
    """

    # --- 1. 基础部分：比较当前城市到候选城市的直线距离和未访问城市的平均距离 ---
    n_cities = len(unvisited)
    total_distance = 0.0
    for i in range(n_cities):
        total_distance += distances_row[unvisited[i]]

    avg_distance = total_distance / n_cities
    direct_distance = distances_row[current_idx]

    if direct_distance < avg_distance:
        priority_score = -(direct_distance - avg_distance)
    else:
        priority_score = -(direct_distance + avg_distance)

    # --- 2. 特殊条件加成：优先考虑起点相关或特定较近城市 ---
    if start_idx == current_idx:
        priority_score += 10
    elif start_idx == candidate_idx:
        priority_score += 5
    elif direct_distance < avg_distance / 2:
        priority_score += 3

    # --- 3. 路径优化逻辑：倾向于距离小于平均距离的城市 ---
    path_optimization_score = 0
    for i in range(n_cities):
        if distances_row[unvisited[i]] < avg_distance:
            path_optimization_score += 1
    priority_score += path_optimization_score

    # --- 4. 环路闭合优化：优先靠近起点的城市 ---
    if candidate_idx == start_idx:
        priority_score += 7
    elif abs(candidate_idx - start_idx) == 1 or abs(candidate_idx - start_idx) == n_cities - 1:
        priority_score += 4

    return priority_score

    return -total_distance  # 负号，路径越短priority越高
def tsp_solve(dist: np.ndarray, start: int = 0) -> list[int]:
    n = dist.shape[0]
    visited: set[int] = {start}
    tour: List[int] = [start]

    for _ in range(n - 1):
        # Find unvisited cities
        unvisited = _get_unvisited(n, visited)

        best_candidate = -1
        best_priority = -np.inf

        for candidate in unvisited:
            # Distances from candidate city to all cities
            distances_row = dist[candidate]

            # Compute priority scores for all unvisited cities
            priority = tsp_priority(
                start_idx=start,
                current_idx=tour[-1],
                candidate_idx=candidate,
                unvisited=unvisited,
                distances_row=distances_row
            )
            if priority>best_priority:
                best_candidate = candidate
                best_priority = priority

        # Update tour
        tour.append(best_candidate)
        visited.add(best_candidate)
    return tour # 不含重复首尾

# @funsearch.run
def evaluate(instances: dict) -> float:
    total_costs, total_times = [], []
    compare_with_optimal = True
    summary = []

    for name, instance in instances.items():
        distances = instance["distances"]

        start_time = time.perf_counter()
        route = tsp_solve(distances)
        elapsed = time.perf_counter() - start_time

        cost = tsp_evaluate(route, distances)
        total_costs.append(cost)
        total_times.append(elapsed)

        # Optimal reference (optional)
        opt_len: float | None = None
        if "optimal_tour" in instance and instance["optimal_tour"] is not None:
            tour = np.asarray(instance["optimal_tour"]).flatten()
            idx = np.arange(len(tour))
            opt_len = float(dist[tour, tour[(idx + 1) % len(tour)]].sum())

        if opt_len is not None:
            approx = cost / opt_len
            print(f"    路径 = {cost:.0f}, 最优 = {opt_len:.0f}, 近似比 = {approx:.4f}, 时间 = {elapsed:.3f}s")
        else:
            print(f"    路径 = {cost:.0f}, 时间 = {elapsed:.3f}s")
            print(f"    路线顺序 = {route}")
    # total_costs = np.array(total_costs)
    # weights = total_costs + 1e-6  # 每个cost作为自己的权重
    # weighted_mean = np.sum(total_costs * weights) / np.sum(weights)

    return -np.mean(total_costs)
# '''

instances = {
    'berlin52': dataset_to_eval['berlin52']['section1']  # 'berlin52', 'eil76' 'kroA100' 'pr124' 'd198' 'lin318' 'pcb442'
}

score = evaluate(instances)
print('Score =', score)
# # # 直接一次性准备所有实例
# instances = {}

# for name in dataset_to_eval.keys():
#     instances[name] = dataset_to_eval[name]['section1']

# # 然后一次性 evaluate 所有！
# score = evaluate(instances)
# print(f'Total weighted score across all instances = {score:.4f}')

    路径 = 32716, 时间 = 0.295s
    路线顺序 = [0, np.int64(33), np.int64(36), np.int64(34), np.int64(38), np.int64(35), np.int64(39), np.int64(37), np.int64(43), np.int64(47), np.int64(48), np.int64(23), np.int64(4), np.int64(31), np.int64(45), np.int64(21), np.int64(14), np.int64(5), np.int64(15), np.int64(49), np.int64(24), np.int64(30), np.int64(3), np.int64(19), np.int64(44), np.int64(22), np.int64(17), np.int64(18), np.int64(11), np.int64(42), np.int64(28), np.int64(27), np.int64(2), np.int64(25), np.int64(20), np.int64(40), np.int64(26), np.int64(29), np.int64(7), np.int64(50), np.int64(46), np.int64(9), np.int64(41), np.int64(12), np.int64(16), np.int64(10), np.int64(8), np.int64(6), np.int64(13), np.int64(32), np.int64(1), np.int64(51)]
Score = -32716.369827436505


## 5. Start FunSearch
Please note that in jupyter notebook the following code will fail. This is because juypter does not support multiprocessing. Colab backend supports multiprocessing.

In [8]:
from implementation import funsearch
from implementation import config
import dataclasses

# It should be noted that the if __name__ == '__main__' is required.
# Because the inner code uses multiprocess evaluation.
if __name__ == '__main__':
    class_config = config.ClassConfig(llm_class=LLMAPI, sandbox_class=Sandbox)

    # default setting
    config = config.Config(samples_per_prompt=4, evaluate_timeout_seconds=30)
    global_max_sample_num = 500  # if it is set to None, funsearch will execute an endless loop

    # test setting
    #config = config.Config(samples_per_prompt=1, evaluate_timeout_seconds=30,programs_database=config.ProgramsDatabaseConfig(num_islands=1))
    #global_max_sample_num = 2

    funsearch.main(
        specification=specification,
        inputs=dataset_to_eval,
        config=config,
        max_sample_nums=global_max_sample_num,
        class_config=class_config,
        log_dir='../logs/funsearch_llm_api'
    )


import numpy as np
import time
from pathlib import Path
from typing import Dict, List, Tuple
 [Function(name='tsp_evaluate', args='route: list[int], distances: np.ndarray', body='    return (\n        sum(distances[route[i], route[i + 1]] for i in range(len(route) - 1))\n        + distances[route[-1], route[0]]\n    )', return_type='float', docstring=None, score=None, global_sample_nums=None, sample_time=None, evaluate_time=None), Function(name='_get_unvisited', args='n: int, visited: set[int]', body='    mask = np.ones(n, dtype=bool)\n    mask[list(visited)] = False\n    return np.nonzero(mask)[0]', return_type='np.ndarray', docstring=None, score=None, global_sample_nums=None, sample_time=None, evaluate_time=None), Function(name='_arg_best', args='prio: np.ndarray', body='    return int(np.argmax(prio))', return_type='int', docstring=None, score=None, global_sample_nums=None, sample_time=None, evaluate_time=None), Function(name='tsp_priority', args='start_idx: int, current_idx: int, 

INFO:absl:Best score of island 0 increased to -8980.918279329191
INFO:absl:Best score of island 1 increased to -8980.918279329191
INFO:absl:Best score of island 2 increased to -8980.918279329191
INFO:absl:Best score of island 3 increased to -8980.918279329191
INFO:absl:Best score of island 4 increased to -8980.918279329191
INFO:absl:Best score of island 5 increased to -8980.918279329191
INFO:absl:Best score of island 6 increased to -8980.918279329191
INFO:absl:Best score of island 7 increased to -8980.918279329191
INFO:absl:Best score of island 8 increased to -8980.918279329191
INFO:absl:Best score of island 9 increased to -8980.918279329191


================= Evaluated Function =================
def tsp_priority(start_idx: int, current_idx: int, candidate_idx: int, unvisited: np.ndarray, distances_row: np.ndarray) -> float:
    """
    Calculate priority scores for all candidate cities at once.

    Args:
        start_idx: int: Index of the starting city (In the Traveling Salesman Problem (TSP),
            the solution requires the salesman to return to the starting city after visiting all other cities.).
        current_idx: int: Index of the current city.
        candidate_idx: int: Index of one candidate city (one selected city in unvisited cities).
        unvisited: np.ndarray: Indices of all unvisited cities
        distances_row: np.ndarray: Distances from the candidate city to all other cities 
            (distances_row[current_idx] represents the distance from candidate city to current city)
            (distances_row[unvisited[0]] represents the distance from candidate city to another unvisited city).

    Ret

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


response ChatCompletion(id='chatcmpl-BQwtDV8YT6py9TJXdBXQJZqJg5z9i', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def tsp_priority_v2(start_idx: int, current_idx: int, candidate_idx: int, unvisited: np.ndarray, distances_row: np.ndarray) -> float:\n    current_distance = distances_row[current_idx]\n    nearest_distance = np.inf\n    nearest_idx = -1\n    \n    for idx in unvisited:\n        if idx != candidate_idx:\n            distance = distances_row[idx]\n            if distance < nearest_distance:\n                nearest_distance = distance\n                nearest_idx = idx\n                \n    if nearest_idx == -1:\n        return 0.0\n\n    back_to_start_distance = distances_row[start_idx]\n    \n    total_distance_with_candidate = current_distance + nearest_distance + back_to_start_distance\n    total_distance_without_candidate = current_distance + back_to_start_distance\n\n    if total_distance_with_candidate < total_d

...

response ChatCompletion(id='chatcmpl-BQxLGhLQN0RXgIGtMpCzaHapnYJsy', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def tsp_priority_v3(start_idx: int, current_idx: int, candidate_idx: int, unvisited: np.ndarray, distances_row: np.ndarray) -> float:\n    """More complex version of tsp_priority function."""\n    \n    direct_distance = distances_row[current_idx]\n    total_distance = direct_distance\n    \n    min_distance = np.inf\n    min_idx = -1\n    min_to_start_distance = np.inf\n    min_to_start_idx = -1\n    \n    for idx in unvisited:\n        if distances_row[idx] < min_distance:\n            min_distance = distances_row[idx]\n            min_idx = idx\n            \n        if distances_row[idx] < min_to_start_distance:\n            min_to_start_distance = distances_row[idx]\n            min_to_start_idx = idx\n    \n    total_distance += min_distance\n    \n    if len(unvisited) == 1:\n        total_distance += distances_

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


response ChatCompletion(id='chatcmpl-BQxLJj1vAFnhEPdxZUQmveiFPfWot', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def tsp_priority_v3(start_idx: int, current_idx: int, candidate_idx: int, unvisited: np.ndarray, distances_row: np.ndarray) -> float:\n    """Creative version of TSP priority calculation."""\n\n    n = len(unvisited)\n    total_distance = distances_row[current_idx]\n\n    # Calculate total distance to the nearest unvisited city\n    min_distance = np.inf\n    min_idx = -1\n    for idx in unvisited:\n        if distances_row[idx] < min_distance:\n            min_distance = distances_row[idx]\n            min_idx = idx\n    total_distance += min_distance\n\n    # Calculate total distance to start city at the end\n    if n == 1:\n        total_distance += distances_row[start_idx]\n    else:\n        min_distance_to_start = np.inf\n        for idx in unvisited:\n            if distances_row[idx] < min_distance_to_start:\n 

INFO:httpx:HTTP Request: POST https://api.bltcy.ai/v1/chat/completions "HTTP/1.1 200 OK"


response ChatCompletion(id='chatcmpl-BQxLLejo2QfwbUJeFDamNe7pByBD2', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='def tsp_priority_v3(start_idx: int, current_idx: int, candidate_idx: int, unvisited: np.ndarray, distances_row: np.ndarray) -> float:\n    """Improved version of `tsp_priority_v2`."""\n\n    n = len(unvisited)\n    total_distance = distances_row[current_idx]\n\n    min_distance = np.inf\n    min_idx = -1\n    for idx in unvisited:\n        if distances_row[idx] < min_distance:\n            min_distance = distances_row[idx]\n            min_idx = idx\n\n    total_distance += min_distance\n\n    if n == 1:\n        return distances_row[start_idx]  # Go back to start city\n\n    min_distance_to_start = np.inf\n    for idx in unvisited:\n        if distances_row[idx] < min_distance_to_start:\n            min_distance_to_start = distances_row[idx]\n\n    total_distance += min_distance_to_start\n\n    candidate_distances = d

| Instance | Number of Cities | Optimal Tour Length |
|:---------|:-----------------|:--------------------|
| berlin52 | 52 | 7542 |
| eil76 | 76 | 538 |
| kroA100 | 100 | 21282 |
| pr124 | 124 | 59030 |
| d198 | 198 | 15780 |
| lin318 | 318 | 42029 |
| pcb442 | 442 | 50778 |


# Result Recording

## berlin52

Score : -8165.076443810632

Sample time : 2.941274106502533

Evaluate time: 0.806704044342041 Sample orders: 101

In [ ]:
def tsp_priority(
    start_idx: int,
    current_idx: int,
    candidate_idx: int,
    unvisited: List[int],
    distances_row: np.ndarray
) -> float:
    """
    Calculate the priority score for choosing candidate_idx as the next city.
    Higher score means higher priority.
    """

    # --- 1. 基础部分：比较当前城市到候选城市的直线距离和未访问城市的平均距离 ---
    n_cities = len(unvisited)
    total_distance = 0.0
    for i in range(n_cities):
        total_distance += distances_row[unvisited[i]]

    avg_distance = total_distance / n_cities
    direct_distance = distances_row[current_idx]

    if direct_distance < avg_distance:
        priority_score = -(direct_distance - avg_distance)
    else:
        priority_score = -(direct_distance + avg_distance)

    # --- 2. 特殊条件加成：优先考虑起点相关或特定较近城市 ---
    if start_idx == current_idx:
        priority_score += 10
    elif start_idx == candidate_idx:
        priority_score += 5
    elif direct_distance < avg_distance / 2:
        priority_score += 3

    # --- 3. 路径优化逻辑：倾向于距离小于平均距离的城市 ---
    path_optimization_score = 0
    for i in range(n_cities):
        if distances_row[unvisited[i]] < avg_distance:
            path_optimization_score += 1
    priority_score += path_optimization_score

    # --- 4. 环路闭合优化：优先靠近起点的城市 ---
    if candidate_idx == start_idx:
        priority_score += 7
    elif abs(candidate_idx - start_idx) == 1 or abs(candidate_idx - start_idx) == n_cities - 1:
        priority_score += 4

    return priority_score
